In [1]:
import pandas as pd

In [2]:
order_data = pd.read_csv("first_25000_rows.csv")
order_data.head()

,ts_recv,ts_event,rtype,publisher_id,instrument_id,action,side,depth,price,size,...,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09,symbol
0,2024-10-21T11:54:29.221230963Z,2024-10-21T11:54:29.221064336Z,10,2,38,C,B,1,233.62,2,...,155,1,7,233.25,234.13,55,400,2,1,AAPL
1,2024-10-21T11:54:29.223936626Z,2024-10-21T11:54:29.223769812Z,10,2,38,A,B,0,233.67,2,...,155,1,7,233.25,234.13,55,400,2,1,AAPL
2,2024-10-21T11:54:29.225196809Z,2024-10-21T11:54:29.225030400Z,10,2,38,A,B,0,233.67,3,...,155,1,7,233.25,234.13,55,400,2,1,AAPL
3,2024-10-21T11:54:29.712600612Z,2024-10-21T11:54:29.712434212Z,10,2,38,A,B,2,233.52,200,...,155,1,7,233.25,234.13,55,400,2,1,AAPL
4,2024-10-21T11:54:29.764839221Z,2024-10-21T11:54:29.764673165Z,10,2,38,C,B,2,233.52,200,...,155,1,7,233.25,234.13,55,400,2,1,AAPL


In [3]:
order_data.describe()

,rtype,publisher_id,instrument_id,depth,price,size,flags,ts_in_delta,sequence,bid_px_00,...,bid_sz_08,ask_sz_08,bid_ct_08,ask_ct_08,bid_px_09,ask_px_09,bid_sz_09,ask_sz_09,bid_ct_09,ask_ct_09
count,5000.0,5000.0,5000.0,5000.000000,5000.000000,5000.000000,5000.000000,5.000000e+03,5.000000e+03,5000.000000,...,5000.000000,5000.000000,5000.00000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,10.0,2.0,38.0,1.407600,233.806564,153.169200,128.031600,1.707528e+05,1.344078e+07,233.755936,...,89.929600,259.928600,1.38500,1.886200,233.364424,234.464900,68.382200,185.662200,1.434200,2.097200
std,0.0,0.0,0.0,1.455158,0.170972,129.459973,15.802283,5.433114e+04,1.254097e+06,0.112182,...,135.718278,450.030627,0.50242,1.341943,0.076242,0.232418,89.704575,344.015312,0.504104,1.460198
min,10.0,2.0,38.0,0.000000,232.600000,1.000000,0.000000,1.655530e+05,1.140585e+07,233.500000,...,1.000000,1.000000,1.00000,1.000000,233.200000,233.680000,1.000000,1.000000,1.000000,1.000000
25%,10.0,2.0,38.0,0.000000,233.690000,29.000000,130.000000,1.661180e+05,1.235669e+07,233.700000,...,15.000000,60.000000,1.00000,1.000000,233.310000,234.250000,15.000000,5.000000,1.000000,1.000000
50%,10.0,2.0,38.0,1.000000,233.800000,200.000000,130.000000,1.663280e+05,1.327258e+07,233.760000,...,100.000000,105.000000,1.00000,1.000000,233.390000,234.430000,55.000000,63.000000,1.000000,1.000000
75%,10.0,2.0,38.0,2.000000,233.920000,200.000000,130.000000,1.666125e+05,1.456848e+07,233.850000,...,110.000000,400.000000,2.00000,2.000000,233.400000,234.590000,110.000000,105.000000,2.000000,4.000000
max,10.0,2.0,38.0,9.000000,235.000000,1200.000000,130.000000,2.215188e+06,1.556924e+07,233.980000,...,1207.000000,2000.000000,3.00000,7.000000,233.660000,235.030000,1109.000000,2000.000000,3.000000,7.000000


In [14]:
# Initialize OFI column
order_data['best_level_ofi'] = 0

In [31]:
def get_ofi(dtfrm, i, m):

    bid_px_m = f'bid_px_0{m}'
    bid_sz_m = f'bid_sz_0{m}'
    ask_px_m = f'ask_px_0{m}'
    ask_sz_m = f'ask_sz_0{m}'


    # Get bid and ask prices and sizes at time i and i-1
    bid_px_now = dtfrm.at[i, bid_px_m]
    bid_px_prev = dtfrm.at[i - 1, bid_px_m]
    bid_sz_now = dtfrm.at[i, bid_sz_m]
    bid_sz_prev = dtfrm.at[i - 1, bid_sz_m]

    ask_px_now = dtfrm.at[i, ask_px_m]
    ask_px_prev = dtfrm.at[i - 1, ask_px_m]
    ask_sz_now = dtfrm.at[i, ask_sz_m]
    ask_sz_prev = dtfrm.at[i - 1, ask_sz_m]

    # Compute bid-side order flow
    if bid_px_now > bid_px_prev:
        of_bid = bid_sz_now
    elif bid_px_now == bid_px_prev:
        of_bid = bid_sz_now - bid_sz_prev
    else:
        of_bid = -bid_sz_prev

    # Compute ask-side order flow
    if ask_px_now > ask_px_prev:
        of_ask = -ask_sz_now
    elif ask_px_now == ask_px_prev:
        of_ask = ask_sz_now - ask_sz_prev
    else:
        of_ask = ask_sz_prev

    # compute avg depth
    avg_depth_m = (bid_sz_now + ask_sz_now) * 0.5

    return of_bid - of_ask, avg_depth_m


In [32]:
# Loop through rows
for i in range(1, len(order_data)):

    order_data.loc[i, 'best_level_ofi'], _ = get_ofi(order_data, i, 0)


In [44]:
order_data['best_level_ofi']

0        0
1        2
2        3
3        0
4        0
        ..
4995     0
4996     0
4997     0
4998     0
4999   -20
Name: best_level_ofi, Length: 5000, dtype: int64

In [ ]:
multi_level_ofi_cols = []
# initialize normalized ofi columns from 0 to 9
for m in range(10):
    order_data[f'ofi_m{m}'] = 0.0
    multi_level_ofi_cols.append(f'ofi_m{m}')


In [38]:
for i in range(1, len(order_data)):

    depth_sum = 0

    for m in range(10):
        ofi, avg_depth_m = get_ofi(order_data, i, m)
        depth_sum += avg_depth_m
        avg_depth = depth_sum / (m + 1)

        value = round(ofi / avg_depth, 4) if avg_depth != 0 else 0.0
        order_data.loc[i, f'ofi_m{m}'] = value

In [39]:

order_data[multi_level_ofi_cols].describe()

,ofi_m0,ofi_m1,ofi_m2,ofi_m3,ofi_m4,ofi_m5,ofi_m6,ofi_m7,ofi_m8,ofi_m9
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,-0.300742,-0.071242,0.008987,-0.006670,-0.017395,-0.011458,-0.022965,-0.025522,-0.021792,-0.025616
std,5.021537,1.271547,1.324161,1.500039,1.249796,1.114818,1.145465,1.201228,1.116506,1.239610
min,-200.000000,-20.779200,-21.818200,-6.971800,-8.324100,-6.656200,-8.717300,-9.321300,-10.086900,-10.860700
25%,0.000000,0.000000,-0.007425,-0.003700,-0.003400,-0.004400,-0.035200,-0.033450,-0.033125,-0.007225
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.063850,0.004700,0.003700,0.004700,0.033100,0.033600,0.031850,0.007600
max,19.200000,6.349200,9.677400,7.044400,6.527900,6.245800,6.393500,6.728900,6.698900,7.295300


## Integrated OFI using PCA

Importing necessary libraries for PCA and other calculations

In [45]:
from sklearn.decomposition import PCA
import numpy as np

Getting the PCA for all the multi level OFI and calculating the integrated OFI using the first principal component vector by normalize it and multiplying with the multi level OFIs.

In [ ]:
X = order_data[multi_level_ofi_cols].dropna().values  # shape: (n_samples, 10)

pca = PCA(n_components=1)
pca.fit(X)

# First principal component vector
w1 = pca.components_[0]           # shape: (10,) row-vector which is equivalent to w1^T
w1_normalized = w1 / np.sum(np.abs(w1))  # normalize by L1 norm

order_data['integrated_ofi'] = np.dot(order_data[multi_level_ofi_cols], w1_normalized)
order_data['integrated_ofi'] = order_data['integrated_ofi'].round(4)

In [49]:
order_data['integrated_ofi']

0       0.0000
1       0.0098
2       0.0145
3       0.0381
4      -0.0516
         ...  
4995    0.1125
4996   -0.0265
4997   -0.1092
4998    0.0856
4999   -2.6896
Name: integrated_ofi, Length: 5000, dtype: float64

### Log returns and summary stats  

Getting the log returns of the mid price of best bid and ask

In [50]:
order_data['log_returns'] = np.log((order_data['bid_px_00'] + order_data['ask_px_00']) / 2).diff()

In [55]:
# Select the multi level ofi cols and log return col
summary_cols = multi_level_ofi_cols + ['integrated_ofi', 'log_returns']  
summary_df = order_data[summary_cols].dropna()

# Create summary stats table
summary_stats = pd.DataFrame({
    'Mean': summary_df.mean(),
    'Std': summary_df.std(),
    'Skewness': summary_df.skew(),
    'Kurtosis': summary_df.kurtosis(),
    '10%': summary_df.quantile(0.10),
    '25%': summary_df.quantile(0.25),
    '50%': summary_df.quantile(0.50),
    '75%': summary_df.quantile(0.75),
    '90%': summary_df.quantile(0.90)
}).round(2)

# Optional: rename index to match paper style
summary_stats.index = [f'ofi{m+1},(1m)' for m in range(10)] + ['ofiI,(1m)', 'r(1m)']

summary_stats.head(12)

,Mean,Std,Skewness,Kurtosis,10%,25%,50%,75%,90%
"ofi1,(1m)",-0.30,5.02,-27.75,958.89,-0.22,0.00,0.0,0.00,0.22
"ofi2,(1m)",-0.07,1.27,-3.91,44.51,-0.96,0.00,0.0,0.00,0.97
"ofi3,(1m)",0.01,1.32,-2.19,35.44,-0.86,-0.01,0.0,0.06,1.16
"ofi4,(1m)",-0.01,1.50,-0.35,4.92,-1.43,-0.00,0.0,0.00,1.45
"ofi5,(1m)",-0.02,1.25,-0.37,9.05,-1.05,-0.00,0.0,0.00,0.95
"ofi6,(1m)",-0.01,1.11,-0.09,8.31,-0.88,-0.00,0.0,0.00,0.77
"ofi7,(1m)",-0.02,1.15,-0.60,8.44,-1.12,-0.04,0.0,0.03,1.05
"ofi8,(1m)",-0.03,1.20,-1.69,21.34,-0.51,-0.03,0.0,0.03,0.50
"ofi9,(1m)",-0.02,1.12,-1.84,26.49,-0.52,-0.03,0.0,0.03,0.51
"ofi10,(1m)",-0.03,1.24,-2.32,32.65,-0.52,-0.01,0.0,0.01,0.52
